<a href="https://colab.research.google.com/github/Dimble-Scaria/Tackling-counterfeit-products-using-CNN/blob/main/Image%20Processing%20using%20Autoencoder%20Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import skimage
import tensorflow as tf

In [3]:
from google.colab import drive                #Mounting with google drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [12]:
image_list = []
for filename in glob.glob('/content/gdrive/MyDrive/capstone_project/tom/*.jpg'):
    img_color = cv2.imread(filename,1)
    image_list.append(img_color)

In [51]:
image_list = []
import PIL
import matplotlib.pyplot as plt
for filename in glob.glob('/content/gdrive/MyDrive/capstone_project/tom/*.jpg'):
  img = PIL.Image.open(filename)
  img = img.resize((50, 50))
  gray_img = img.convert("L")
  image_list.append(gray_img)
#plt.imshow(gray_img, cmap='gray')

In [75]:
image_pixel_list=[]
for k in image_list:
  image=[]
  for i in range(50):
    row=[]
    for j in range(50):
      pixel=np.array(k.getpixel((i, j)))
      row.append(pixel)
    image.append(np.array(row))
  image_pixel_list.append(np.array(image))

In [79]:
image_pixel_array=np.array(image_pixel_list) #Images are read and stored as an array of images of size 50X50
image_pixel_array.shape

(1930, 50, 50)

In [23]:
gray_image_list=[]
#for i in range(500):
  k=skimage.color.rgb2gray(image_list[i])
  gray_image_list.append(k)
#gray_image_array=np.array(gray_image_list)

In [83]:
#gray_image_list=pd.DataFrame(gray_image_list)
#x_train = gray_image_list.astype('float32') / 255.
image_pixel_array=np.array(image_pixel_list) #Images are read and stored as an array of images of size 50X50
x_train = image_pixel_array
x_test = image_pixel_array

In [85]:
inputs = tf.keras.Input(shape=(50, 50, 1), name='input_layer')
# Conv Block 1 -> BatchNorm->leaky Relu
encoded = tf.keras.layers.Conv2D(32, kernel_size=3, strides= 1, padding='same', name='conv_1')(inputs)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_1')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_1')(encoded)
# Conv Block 2 -> BatchNorm->leaky Relu
encoded = tf.keras.layers.Conv2D(64, kernel_size=3, strides= 2, padding='same', name='conv_2')(encoded)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_2')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_2')(encoded)
# Conv Block 3 -> BatchNorm->leaky Relu
encoded = tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', name='conv_3')(encoded)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_3')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_3')(encoded)
#Decoder
# DeConv Block 1-> BatchNorm->leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(64, 3, strides= 1, padding='same',name='conv_transpose_1')(encoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_4')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_4')(decoded)
# DeConv Block 2-> BatchNorm->leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(64, 3, strides= 2, padding='same', name='conv_transpose_2')(decoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_5')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_5')(decoded)
# DeConv Block 3-> BatchNorm->leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(32, 3, 2, padding='same', name='conv_transpose_3')(decoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_6')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_6')(decoded)
# output
outputs = tf.keras.layers.Conv2DTranspose(1, 3, 1,padding='same', activation='sigmoid', name='conv_transpose_4')(decoded)

In [92]:

def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0))

In [93]:
autoencoder = tf.keras.Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005)
autoencoder.compile(optimizer=optimizer, loss=SSIMLoss)

In [95]:
hist=autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test)
                )

Epoch 1/10


ValueError: ignored

In [9]:
 gray_image_list[0]= gray_image_list[0].astype('float32') / 255
 np.reshape(gray_image_list[0], (len(gray_image_list[0]), 28, 28, 1))

ValueError: ignored